In [201]:
%matplotlib qt5
import sys
import copy
from pathlib import Path
import numpy as np
import scipy.signal
import scipy.ndimage
import sunpy.io.fits
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.optimize import curve_fit
from scipy.special import softmax
from scipy.signal import medfilt2d
from tqdm import tqdm
import h5py

In [2]:
base_path = Path(
    '/Volumes/HarshHDD-Data/Documents/CourseworkRepo/Polcal/20220323'
)

In [3]:
write_path = base_path / 'Processed'

In [4]:
raw_flat_filenames = ['DETECTOR_1_115045_FLAT.fits']

In [5]:
raw_dark_frame = 'DETECTOR_1_114550_DARK.fits'

In [6]:
def get_dark_frame():

    data, header = sunpy.io.fits.read(base_path / 'Darks' / raw_dark_frame)[0]

    mean_dark = np.mean(data, 0)

    sunpy.io.fits.write(write_path / 'dark_master.fits', mean_dark, dict(), overwrite=True)

    return mean_dark

In [7]:
mean_dark = get_dark_frame()

In [12]:
flat_data, header = sunpy.io.fits.read(base_path / 'Flats' / raw_flat_filenames[0])[0]

In [13]:
flat_data.shape, flat_data.dtype

((90, 512, 512), dtype('uint16'))

In [14]:
flat_data = flat_data.astype(np.float64) - mean_dark[np.newaxis, :, :]

In [16]:
plt.imshow(flat_data[50], cmap='gray', origin='lower')

In [17]:
mean_flat = np.mean(flat_data, 0)

In [27]:
plt.imshow(mean_flat, cmap='gray', origin='lower')

In [19]:
mean_flat.min()

194.08666666666738

In [61]:
plt.figure('Select 50 points on bright fringe')

plt.imshow(np.log(mean_flat), cmap='gray', origin='lower')

point = np.array(plt.ginput(50, 600))

In [62]:
point = point.astype(np.int64)

In [63]:
a, b, c, d, e = np.polyfit(point[:, 1], mean_flat[point[:, 1], point[:, 0]], 4)

y = a * np.arange(512)**4 + b * np.arange(512)**3 + c * np.arange(512)**2 + d * np.arange(512) + e

plt.plot(np.arange(512), y)
plt.plot(point[:, 1], mean_flat[point[:, 1], point[:, 0]])

In [64]:
y/= y.max()
plt.imshow(mean_flat / y[:, np.newaxis], cmap='gray', origin='lower')

In [65]:
plt.imshow(flat_data[0] / y[:, np.newaxis], cmap='gray', origin='lower')

In [66]:
nfdata = flat_data / y[np.newaxis, :, np.newaxis]

In [67]:
nmean = mean_flat / y[:, np.newaxis]

In [68]:
plt.imshow(nmean, cmap='gray', origin='lower')

In [70]:
plt.figure('Select 50 points on bright fringe of one of the beams')

plt.imshow(nmean, cmap='gray', origin='lower')

point = np.array(plt.ginput(50, 600))

In [71]:
point = point.astype(np.int64)

In [75]:
f, g, h = np.polyfit(point[:, 0], nmean[point[:, 1], point[:, 0]], 2)

z = f * np.arange(512)**2 + g * np.arange(512) + h

plt.plot(np.arange(512), z)
plt.plot(point[:, 0], nmean[point[:, 1], point[:, 0]])

In [78]:
z/= z.max()
plt.imshow(nmean / z[np.newaxis, :], cmap='gray', origin='lower')

In [79]:
nnmean = nmean / z[np.newaxis, :]

In [80]:
nnfdata = nfdata / z[np.newaxis, np.newaxis, :]

In [81]:
plt.imshow(nnfdata[0], cmap='gray', origin='lower')

In [84]:
medfiltered_better_flat = np.zeros_like(nnfdata)

In [85]:
medfiltered_better_flat.shape

(90, 512, 512)

In [86]:
for i in range(medfiltered_better_flat.shape[0]):
    medfiltered_better_flat[i] = medfilt2d(nnfdata[i], kernel_size=3)

In [87]:
plt.imshow(medfiltered_better_flat[0], cmap='gray', origin='lower')

In [97]:
D_matrix = medfiltered_better_flat.reshape(90, 512 * 512).T

In [98]:
D_matrix.shape

(262144, 90)

In [99]:
pca = PCA()

In [100]:
pca = pca.fit(D_matrix)

In [101]:
lambda_j = pca.explained_variance_

In [102]:
lambda_j.shape

(90,)

In [103]:
plt.plot(lambda_j)

In [104]:
ej_matrix = np.zeros_like(D_matrix)
U_matrix = pca.components_

In [107]:
for j in range(ej_matrix.shape[1]):
    ej_matrix_temp = np.zeros(ej_matrix.shape[0])
    for i in range(D_matrix.shape[1]):
        ej_matrix_temp += D_matrix[:, i] * U_matrix[i, j]
    ej_matrix[:, j] = np.sqrt(lambda_j[j]) * ej_matrix_temp

In [111]:
plt.imshow(ej_matrix[:, 10].reshape(512, 512), cmap='gray', origin='lower')

In [112]:
C_matrix = np.dot(np.dot(np.linalg.inv(np.dot(ej_matrix.T, ej_matrix)), ej_matrix.T), D_matrix)

In [113]:
C_matrix.shape

(90, 90)

In [114]:
plt.imshow(C_matrix, cmap='gray', origin='lower')

In [116]:
plt.imshow(nnmean[300:500, 50:250], cmap='gray', origin='lower')

In [117]:
sel_region = nnmean[300:500, 50:250]

In [180]:
corr = scipy.signal.fftconvolve(sel_region, sel_region[::-1,::-1], mode='same')

In [181]:
corr.shape, np.sum(corr), corr.dtype

((200, 200), 1.2349445897996704e+17, dtype('float64'))

In [165]:
plt.imshow(np.abs(corr), cmap='gray', origin='lower')

In [190]:
new_ej_matrix = ej_matrix.copy()

In [193]:
new_ej_matrix.shape

(262144, 90)

In [194]:
write_path = Path('/Volumes/HarshHDD-Data/Documents/CourseworkRepo/Polcal/Fringe Removal')

In [196]:
sunpy.io.fits.write(write_path / 'medfiltered_better_flat.fits', medfiltered_better_flat, dict())

In [198]:
sunpy.io.fits.write(write_path / 'nnmean.fits', nnmean, dict())

In [199]:
sunpy.io.fits.write(write_path / 'D_matrix.fits', D_matrix, dict())
sunpy.io.fits.write(write_path / 'U_matrix.fits', U_matrix, dict())
sunpy.io.fits.write(write_path / 'lambda_j.fits', lambda_j, dict())
sunpy.io.fits.write(write_path / 'ej_matrix.fits', ej_matrix, dict())

In [200]:
sunpy.io.fits.write(write_path / 'C_matrix.fits', C_matrix, dict())

In [203]:
f = h5py.File(write_path / 'Data.h5', 'w')
f['medfiltered_better_flat'] = medfiltered_better_flat
f['nnmean'] = nnmean
f['D_matrix'] = D_matrix
f['U_matrix'] = U_matrix
f['lambda_j'] = lambda_j
f['ej_matrix'] = ej_matrix
f['C_matrix'] = C_matrix
f.close()

In [192]:
if tp is not None:
    del tp
tp = tqdm(total=D_matrix.shape[1] * (D_matrix.shape[1]-1) * 18)
score = np.mean(np.abs(corr))
max_niter = 2
k = 0
try:
    for i in range(D_matrix.shape[1]):
        for j in range(i+1, D_matrix.shape[1], 1):
            for deg_theta in np.arange(-90, 90, 10) :
                %timeit old_ei_xy = new_ej_matrix[:, i].copy()
                %timeit old_ej_xy = new_ej_matrix[:, j].copy()
                %timeit new_ej_matrix[:, i] = np.cos(theta) * old_ei_xy + np.sin(theta) * old_ej_xy
                %timeit new_ej_matrix[:, j] = -np.sin(theta) * old_ei_xy + np.cos(theta) * old_ej_xy
                %timeit new_D = np.dot(new_ej_matrix, C_matrix)
                %timeit mean_new_D = np.mean(new_D.T.reshape(90, 512, 512), 0)
                %timeit ns = np.mean(np.abs(scipy.signal.fftconvolve(sel_region, mean_new_D[300:500, 50:250][::-1, ::-1], mode='same')))
                if ns < score:
                    score = ns
                    tp.set_postfix(score=score)
                else:
                    %timeit new_ej_matrix[:, i] = old_ei_xy
                    %timeit new_ej_matrix[:, j] = old_ej_xy
                tp.update(1)
                k += 1
                if k > max_niter:
                    raise Exception
except:
    pass
            









  0%|          | 0/144180 [00:00<?, ?it/s]

4.93 ms ± 699 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.53 ms ± 200 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.04 ms ± 497 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.15 ms ± 550 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
266 ms ± 7.43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
25.9 ms ± 5.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)










  0%|          | 0/144180 [00:28<?, ?it/s, score=1.77e+12]







  0%|          | 1/144180 [00:28<1128:07:04, 28.17s/it, score=1.77e+12]

7.42 ms ± 1 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.22 ms ± 151 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.25 ms ± 749 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5.85 ms ± 332 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5.87 ms ± 426 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
300 ms ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
20.9 ms ± 568 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
6.96 ms ± 611 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.86 ms ± 300 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)










  0%|          | 2/144180 [01:00<1232:48:30, 30.78s/it, score=1.77e+12]

3.53 ms ± 26.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5.21 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.12 ms ± 304 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.42 ms ± 941 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.8 ms ± 707 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
313 ms ± 17.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
27.2 ms ± 6.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
7.75 ms ± 650 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.07 ms ± 310 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)










  0%|          | 3/144180 [01:37<1343:15:37, 33.54s/it, score=1.77e+12]

3.91 ms ± 313 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [170]:
plt.imshow(mean_new_D, cmap='gray', origin='lower')